# **Analysis of Launch Site Locations Using Folium**

The success rate of rocket launches may be influenced by various factors, including payload mass, orbit type, and other operational considerations. Additionally, the geographical location and proximity of a launch site—specifically, the initial position of rocket trajectories—may also play a crucial role. The identification of an optimal site for rocket launches necessitates the evaluation of multiple factors, some of which may be discerned through an analysis of existing launch site locations.

In previous exploratory data analysis labs, the SpaceX launch dataset was visualised using `matplotlib` and `seaborn`, revealing preliminary correlations between launch site locations and success rates. In this lab, a more interactive approach to visual analytics will be undertaken using `Folium`.

## Objectives

This lab comprises the following tasks:

* **TASK 1:** Mark all launch sites on a map  
* **TASK 2:** Indicate successful and failed launches for each site on the map  
* **TASK 3:** Compute the distances between each launch site and its surrounding locations  

Upon completion of these tasks, geographical patterns associated with launch sites should become more evident.

First, the required Python packages for this lab will be imported:



In [1]:
# Geospatial visualization library
import folium

# Library for web scrapping
# import wget # Only required if want to work with coursera IBM dataset
#from bs4 import BeautifulSoup

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Standard libraries for handling dates, file operations, and serialization
import datetime
import os
import pickle

# SpaceX-specific module (custom or third-party)
import spacex

# Importing directory paths from SpaceX config
from spacex.config import RAW_DATA_DIR, INTERIM_DATA_DIR, PROCESSED_DATA_DIR, FIGURES_DIR

# Display settings for pandas DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
from folium.plugins import MarkerCluster  # Import folium MarkerCluster plugin
from folium.plugins import MousePosition  # Import folium MousePosition plugin
from folium.features import DivIcon       # Import folium DivIcon plugin

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


In [3]:
spacex_pkl_file = os.path.join(INTERIM_DATA_DIR, 'dataset_part_2.pkl')
spacex_df=pd.read_pickle(spacex_pkl_file)

Now, you can take a look at what are the coordinates for each site.


In [4]:
# Select relevant sub-columns: `Launch_Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch_Site', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['Launch_Site'], as_index=False, observed=False).first()
launch_sites_df = launch_sites_df[['Launch_Site', 'Latitude', 'Longitude']]
launch_sites_df

,Launch_Site,Latitude,Longitude
0,CCSFS SLC-40,28.561857,-80.577366
1,KSC LC-39A,28.608058,-80.603956
2,VAFB SLC-4E,34.632093,-120.610829


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#000ed3', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#ff0000;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small blue circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [7]:
ls_circles=[]
ls_markers=[]
USA_avg_coord=[31,-100]
#print(USA_avg_coord)
#print(launch_sites_df)
# Initial the map
site_map = folium.Map(
    location=USA_avg_coord, 
    zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Latitude, Longitude) values. 
# In addition, add Launch site name as a popup label
for i in range(len(launch_sites_df)):
    ls_coord=[launch_sites_df['Latitude'][i],launch_sites_df['Longitude'][i]]
    ls_circles.append(folium.Circle(ls_coord, 
              radius=100, 
              color='#d35400', 
              fill=True).add_child(folium.Popup(launch_sites_df['Launch_Site'][i])))
    ls_markers.append(folium.map.Marker(ls_coord, 
                                       icon=DivIcon(icon_size=(20,20),
                                                    icon_anchor=(0,0), 
                                                    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df['Launch_Site'][i], 
                                                   )
                                      )
                     )

site_map.add_child(ls_circles[0])
site_map.add_child(ls_markers[0])
site_map.add_child(ls_circles[1])
site_map.add_child(ls_markers[1])
site_map.add_child(ls_circles[2])
site_map.add_child(ls_markers[2])
#site_map.add_child(ls_circles[3])
#site_map.add_child(ls_markers[3])

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.tail(10)

,Launch_Site,Latitude,Longitude,Class
111,CCSFS SLC-40,28.561857,-80.577366,1
112,CCSFS SLC-40,28.561857,-80.577366,1
113,KSC LC-39A,28.608058,-80.603956,1
114,CCSFS SLC-40,28.561857,-80.577366,1
115,KSC LC-39A,28.608058,-80.603956,1
116,CCSFS SLC-40,28.561857,-80.577366,1
117,KSC LC-39A,28.608058,-80.603956,1
118,CCSFS SLC-40,28.561857,-80.577366,1
119,KSC LC-39A,28.608058,-80.603956,1
120,CCSFS SLC-40,28.561857,-80.577366,1


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [9]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [10]:
launch_sites_df

,Launch_Site,Latitude,Longitude
0,CCSFS SLC-40,28.561857,-80.577366
1,KSC LC-39A,28.608058,-80.603956
2,VAFB SLC-4E,34.632093,-120.610829


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch_Site,Latitude,Longitude,Class,marker_color
111,CCSFS SLC-40,28.561857,-80.577366,1,green
112,CCSFS SLC-40,28.561857,-80.577366,1,green
113,KSC LC-39A,28.608058,-80.603956,1,green
114,CCSFS SLC-40,28.561857,-80.577366,1,green
115,KSC LC-39A,28.608058,-80.603956,1,green
116,CCSFS SLC-40,28.561857,-80.577366,1,green
117,KSC LC-39A,28.608058,-80.603956,1,green
118,CCSFS SLC-40,28.561857,-80.577366,1,green
119,KSC LC-39A,28.608058,-80.603956,1,green
120,CCSFS SLC-40,28.561857,-80.577366,1,green


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker=folium.Marker([float(record['Latitude']), float(record['Longitude'])],
                         #icon=folium.Icon(color=record['marker_color'])
                         icon=folium.Icon(color='white', icon_color=record['marker_color']
                        ))
    marker_cluster.add_child(marker)
    #print(index, "    ",record['Latitude'])
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# Add Mouse Position to get the coordinate (Latitude, Longitude) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topleft',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Latitude` and `Longitude` values using the following method:


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [16]:
dic_positions={'position': ['Railway_Station'] ,'Latitude':[28.57695],'Longitude':[-80.58431]}
positions = pd.DataFrame.from_dict(dic_positions)
positions.iloc[0]

position     Railway_Station
Latitude            28.57695
Longitude          -80.58431
Name: 0, dtype: object

In [17]:
# find coordinate of the closet coastline
Lat_1 = 28.57695
Lon_1 = -80.58431
distance_railstation = calculate_distance(launch_sites_df.iloc[0]['Latitude'], launch_sites_df.iloc[0]['Longitude'], Lat_1, Lon_1)
print(calculate_distance(launch_sites_df.iloc[0]['Latitude'], launch_sites_df.iloc[0]['Longitude'], positions.iloc[0]['Latitude'], positions.iloc[0]['Longitude']))
distance_railstation

1.8106467204485905


1.8106467204485905

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [18]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    [Lat_1,# Latitude
    Lon_1],# Longitude
    icon=DivIcon(icon_size=(20,20),
                 icon_anchor=(0,0),
                 html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} km".format(distance_railstation),
                )
)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [19]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates=([Lat_1,Lon_1],[launch_sites_df.iloc[0]['Latitude'], launch_sites_df.iloc[0]['Longitude']])
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(distance_marker)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [20]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


Copyright © 2021 IBM Corporation. All rights reserved.
